<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="600" height="500"></center>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf)!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## `client`
| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `text` | Whether the client's credit is in default |
| `housing` | `text` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `text` | Whether the client has an existing personal loan |

## `campaign`
| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `integer` | Campaign ID |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `text` | Outcome of the previous campaign |
| `campaign_outcome` | `integer` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

## `economics`
| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [111]:
# Start coding...
import pandas as pd
import numpy as np

In [112]:
#Import data

df = pd.read_csv('bank_marketing.csv')
df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [113]:
#Split data

client = df[['client_id','age','job','marital','education','credit_default','housing','loan']]

campaign = df[['client_id','contact','month','day_of_week','duration','campaign','pdays','previous','poutcome', 'y']]

economics = df[['client_id','emp_var_rate','cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed']]



In [114]:
#Rename column
client.rename(columns = {"client_id":"id"},inplace=True)

campaign.rename(columns = {"duration":"contact_duration","previous":"previous_campaign_contacts","y":"campaign_outcome",
                           "poutcome":"previous_outcome","campaign":"number_contacts"},inplace=True)

economics.rename(columns = {"euribor3m":"euribor_three_months","nr_employed":"number_employed"},inplace=True)




In [115]:
#Clean the "education" column
client.education[df.education == "."] = "_"
client.education[df.education == "unknown"] = np.nan

#Remove periods from the "job" column.
client['job'] = client['job'].str.replace(".","")

In [116]:
campaign.head()

,client_id,contact,month,day_of_week,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,telephone,may,mon,261,1,999,0,nonexistent,no
1,1,telephone,may,mon,149,1,999,0,nonexistent,no
2,2,telephone,may,mon,226,1,999,0,nonexistent,no
3,3,telephone,may,mon,151,1,999,0,nonexistent,no
4,4,telephone,may,mon,307,1,999,0,nonexistent,no


In [117]:
#Convert "previous_outcome" and "campaign_outcome" values to binary (1 or 0), including the conversion of "nonexistent" to 0.

campaign["previous_outcome"] = campaign["previous_outcome"].map({"success":"1","failure":"0","nonexistent":"0"})

campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes":"1","no":"0"})

In [118]:
#Create a datetime column called last_contact_date

campaign["last_contact_date"] = campaign["month"].str.capitalize() + "-" + campaign["day_of_week"].str.capitalize() + "-" + "2022"


campaign['last_contact_date']=pd.to_datetime(campaign['last_contact_date'],format="%b-%a-%Y")


In [119]:
#Add a campaign_id as the first column
campaign['campaign_id'] = 1 

campaign = campaign[['campaign_id','client_id', 'contact', 'month', 'day_of_week', 'contact_duration',
       'number_contacts', 'pdays', 'previous_campaign_contacts',
       'previous_outcome', 'campaign_outcome', 'last_contact_date',
       ]]


In [120]:
#Remove any redundant data
campaign.drop(['month', 'day_of_week'],axis=1,inplace=True)

In [121]:
#Save the three subsets to csv files
client.to_csv('client.csv',index=False)
campaign.to_csv('campaign.csv',index=False)
economics.to_csv('economics.csv',index=False)

In [122]:
# Store and print database_design
database_design = {"client":client,"campaign":campaign,"economics":economics}

print(database_design)

{'client':           id  age          job  ... credit_default housing loan
0          0   56    housemaid  ...             no      no   no
1          1   57     services  ...        unknown      no   no
2          2   37     services  ...             no     yes   no
3          3   40        admin  ...             no      no   no
4          4   56     services  ...             no      no  yes
...      ...  ...          ...  ...            ...     ...  ...
41183  41183   73      retired  ...             no     yes   no
41184  41184   46  blue-collar  ...             no      no   no
41185  41185   56      retired  ...             no     yes   no
41186  41186   44   technician  ...             no      no   no
41187  41187   74      retired  ...             no     yes   no

[41188 rows x 8 columns], 'campaign':        campaign_id  client_id  ... campaign_outcome  last_contact_date
0                1          0  ...                0         2022-05-01
1                1          1  ...      